
# Example of High Gamma Filter

Below is a code sample for extracting high gamma power from a raw data file, followed by permutation cluster stats on that high gamma power data


### working version 12/1/23

### try gregs suggestion of using make_data_same to destroy the fixation cross

use window stats with perm testing (0 to 0.5, 0.5 to 1, 0 to 1 sec relative to stim onset)

In [2]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from utils import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans, channel_names_to_indices, \
    filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, \
    perform_permutation_test_within_electrodes, add_accuracy_to_epochs, save_sig_chans_with_reject

import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests

# Directory where your .npy files are saved
npy_directory = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your directory path

# Dictionary to hold the data
acc_array = {}

# Iterate over each file in the directory
for file in os.listdir(npy_directory):
    if file.endswith('.npy'):
        # Construct the full file path
        file_path = os.path.join(npy_directory, file)
        # Load the numpy array from the file
        acc_array[file.split('_')[0]] = np.load(file_path)

# Now you have a dictionary where each key is the subject ID
# and the value is the numpy array of accuracies for that subject.
        
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


use time point cluster stats for determining stimulus significance (old method as of 2/13/24)

updated this one 2/29, once it's tested and works, then turn into a function and delete other cells below

uncomment things and delete the subjects variable once we get the mat shape 3/10

In [183]:
within_times_samples_pos

[0, 4096]

In [181]:
within_times = (-1, 0)
times_length = 0.5
pad_length = 0.5
event = "Stimulus"
raw = good
# Calculate sampling frequency
sfreq = raw.info['sfreq'] #raw.info in function

# get padded within times and times_length
within_times_padded = [within_times[0] - pad_length, within_times[1] + pad_length]
times_length_padded = times_length + 2 * pad_length

# Convert times to samples
within_times_samples = [int(t * sfreq) for t in within_times_padded]
times_length_samples = int((times_length_padded) * sfreq)

# Shift the indices to be positive
shift = abs(within_times_samples[0])
within_times_samples_pos = [s + shift for s in within_times_samples]

trials = trial_ieeg(raw, event, within_times_padded, preload=True, reject_by_annotation=False)

rand_offset_data = []

# Randomly select subsets for each trial
for trial in trials.get_data():
    start_sample = random.randint(within_times_samples_pos[0], within_times_samples_pos[1] - times_length_samples)
    end_sample = start_sample + times_length_samples
    rand_offset_data.append(trial[:, start_sample:end_sample]) #across all channels, grab this time subset

# Reassign data to rand_offset_trials and adjust the times in rand_offset_trials
new_tmin = within_times_padded[0]
new_tmax = new_tmin + times_length_padded
rand_offset_trials = trial_ieeg(raw, event, [new_tmin, new_tmax], preload=True, reject_by_annotation=False)
rand_offset_trials._data = np.array(rand_offset_data)

outliers_to_nan(rand_offset_trials, outliers=10)
HG_base = gamma.extract(rand_offset_trials, copy=False, n_jobs=1)
pad_length_string = f"{pad_length}s" # define pad_length as a string so can use it as input to crop_pad
crop_pad(HG_base, pad_length_string) # need to change this if pad length changes
   

Used Annotations descriptions: ['1', '2', 'Response/c25.0/n25.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount225.0/BlockTrialCount1.0/ReactionTime2149.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime883.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount228.0/BlockTrialCount4.0/ReactionTime766.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount241.0/BlockTrialCount17.0/ReactionTime499.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount257.0/BlockTrialCount33.0/ReactionTim

C:\Users\jz421\AppData\Local\Temp\ipykernel_3924\1775888980.py:26: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for trial in trials.get_data():


0 bad epochs dropped


100%|██████████| 449/449 [02:01<00:00,  3.71it/s]


IndexError: boolean index did not match indexed array along dimension 2; dimension is 3072 but corresponding boolean dimension is 3073

In [ ]:
def to_samples(time_length: Union[str, int], sfreq: float) -> int:
    """Convert a time length to a number of samples.

    Parameters
    ----------
    time_length : str | int
        The time length to convert. If a string, it must be a human-readable
        time, e.g. "10s".
    sfreq : float
        The sampling frequency.

    Returns
    -------
    samples : int
        The number of samples.
    """
    validate_type(time_length, (str, int))
    if isinstance(time_length, str):
        time_length = time_length.lower()
        err_msg = ('filter_length, if a string, must be a '
                   'human-readable time, e.g. "0.7s", or "700ms", not '
                   '"%s"' % time_length)
        low = time_length.lower()
        if low.endswith('us'):
            mult_fact = 1e-6
            time_length = time_length[:-2]
        elif low.endswith('ms'):
            mult_fact = 1e-3
            time_length = time_length[:-2]
        elif low[-1] == 's':
            mult_fact = 1
            time_length = time_length[:-1]
        elif low.endswith('sec'):
            mult_fact = 1
            time_length = time_length[:-3]
        elif low[-1] == 'm':
            mult_fact = 60
            time_length = time_length[:-1]
        elif low.endswith('min'):
            mult_fact = 60
            time_length = time_length[:-3]
        else:
            raise ValueError(err_msg)
        # now get the number
        try:
            time_length = float(time_length)
        except ValueError:
            raise ValueError(err_msg)
        time_length = max(int(np.ceil(time_length * mult_fact * sfreq)), 1)
    time_length = ensure_int(time_length, 'filter_length')
    return time_length

inst = HG_base
copy = False

if copy:
    out = inst.copy()
else:
    out = inst

pad = to_samples(pad, inst.info['sfreq']) / inst.info['sfreq']
out.crop(tmin=inst.tmin + pad, tmax=inst.tmax - pad)

In [153]:
from mne.utils import fill_doc, verbose
import random
from contextlib import redirect_stdout

def trial_ieeg_rand_offset(raw: mne.io.Raw, event: str | list[str, ...], within_times: tuple[float,float], times_length: float, pad_length: float,
               verbose=None, **kwargs) -> mne.Epochs:
    """Epochs data from a mne Raw iEEG instance.

    Takes a mne Raw instance and randomly epochs the data around a specified event, for each instance of the event,
    for a duration of times_length, within a range of within_times.

    Parameters
    ----------
    raw : mne.io.Raw
        The raw data to epoch.
    event : str
        The event to epoch around.
    within_times : tuple[float, float]
        The time window within which to randomly select intervals for each event.
    times_length : float,
        The length of the time intervals to randomly select within `within_times`.
    pad_length : float,
        The length to pad each time interval. Will be removed later.
    %(picks_all)s
    %(reject_epochs)s
    %(flat)s
    %(decim)s
    %(epochs_reject_tmin_tmax)s
    %(detrend_epochs)s
    %(proj_epochs)s
    %(on_missing_epochs)s
    %(verbose)s

    Returns
    -------
    mne.Epochs
        The epoched data.
    """

    sfreq = raw.info['sfreq'] #raw.info in function

    # get padded within times
    within_times_padded = (within_times[0] - pad_length, within_times[1] + pad_length)

    # Convert times to samples
    within_times_samples = [int(t * sfreq) for t in [within_times[0] - pad_length, within_times[1] + pad_length]]
    times_length_samples = int((times_length + 2 * pad_length) * sfreq)

    # Shift the indices to be positive
    shift = abs(within_times_samples[0])
    within_times_samples_pos = [s + shift for s in within_times_samples]

    trials = trial_ieeg(raw, event, within_times_padded, preload=True, reject_by_annotation=False) # get original trial data

    rand_offset_data = []

    # Randomly select subsets for each trial
    for trial in trials.get_data():
        start_sample = random.randint(within_times_samples_pos[0], within_times_samples_pos[1] - times_length_samples)
        end_sample = start_sample + times_length_samples
        rand_offset_data.append(trial[:, start_sample:end_sample]) #across all channels, grab this time subset

    # Reassign data to rand_offset_trials
    rand_offset_trials = trials.copy()
    rand_offset_trials._data = np.array(rand_offset_data)

    return rand_offset_trials

In [155]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from utils import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans, plot_HG_and_stats
import matplotlib.pyplot as plt

from mne.utils import fill_doc, verbose
import random
from contextlib import redirect_stdout

# Define a function to shuffle an array
def shuffle_array(arr):
    np.random.shuffle(arr)
    return arr

def plot_HG_and_stats(sub, task='GlobalLocal', times=(-1, 1.5),
                      within_base_times=(-1, 0), base_times_length=0.5, pad_length = 0.5, LAB_root=None, channels=None, dec_factor=10):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.

    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - times (tuple, optional): A tuple indicating the start and end times for event processing. Defaults to (-1, 1.5).
    - times (tuple [float, float]): The time window to epoch around the event.
    - within_base_times (tuple [float, float]): The time window within which to randomly select intervals for each event, for baseline.
    - base_times_length (float): The length of the time intervals to randomly select within `within_base_times`. 
    - pad_length (float): The length to pad each time interval. Will be removed later.
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - decimation_factor (int, optional): The factor by which to subsample the data. Default is 10, so should be 2048 Hz down to 204.8 Hz.

    This function will process the provided event for a given subject and task.
    High gamma (HG) will be computed, and statistics will be calculated and plotted.
    The results will be saved to output files.
    """

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)
    within_times_duration = abs(within_base_times[1] - within_base_times[0]) #grab the duration as a string for naming

    # Create a baseline EpochsTFR using the stimulus event. For each trial, will randomly grab a segment of duration base_times_length from the within_base_times range. This offsets the fix cross. 6/15.
    trials = trial_ieeg_rand_offset(good, "Stimulus", within_base_times, base_times_length, pad_length, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    pad_length_string = f"{pad_length}s" # define pad_length as a string so can use it as input to crop_pad
    crop_pad(HG_base, pad_length_string) # need to change this if pad length changes
    HG_base.decimate(dec_factor)
    

    output_name_base = f"{base_times_length}sec_within{within_times_duration}sec_randoffset_preStimulusBase_decFactor_{dec_factor}"

    for event in ["Stimulus", "Response"]:
        output_name_event = f'{event}_{output_name_base}'
        times_adj = [times[0] - pad_length, times[1] + pad_length]
        trials = trial_ieeg(good, event, times_adj, preload=True,
                            reject_by_annotation=False)

        outliers_to_nan(trials, outliers=10)
        HG_ev1 = gamma.extract(trials, copy=True, n_jobs=1)
        print("HG_ev1 before crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)
        crop_pad(HG_ev1, pad_length_string) #change this if pad length changes
        print("HG_ev1 after crop_pad: ", HG_ev1.tmin, HG_ev1.tmax)

        HG_ev1.decimate(dec_factor)

        HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')
        HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0)) #axis=0 should be set for actually running this, the axis=2 is just for drift testing.
        HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

        # Save HG_ev1
        HG_ev1.save(f'{save_dir}/{sub}_{output_name_event}_HG_ev1-epo.fif', overwrite=True)

        # Save HG_base (the shuffled version)
        HG_base.save(f'{save_dir}/{sub}_{output_name_event}_HG_base-epo.fif', overwrite=True)

        # Save HG_ev1_rescaled
        HG_ev1_rescaled.save(f'{save_dir}/{sub}_{output_name_event}_HG_ev1_rescaled-epo.fif', overwrite=True)

        # Save HG_ev1_evoke
        HG_ev1_evoke.save(f'{save_dir}/{sub}_{output_name_event}_HG_ev1_evoke-epo.fif', overwrite=True)

        # Save HG_ev1_evoke_rescaled
        HG_ev1_evoke_rescaled.save(f'{save_dir}/{sub}_{output_name_event}_HG_ev1_evoke_rescaled-epo.fif', overwrite=True)

        ###
        print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
        print(f"Shape of HG_base._data: {HG_base._data.shape}")
        
        mat = time_perm_cluster(HG_ev1._data, HG_base._data, 0.05, n_jobs=6, ignore_adjacency=1)

        # Plot the matrix as one figure
        fig, ax = plt.subplots()
        cax = ax.imshow(mat, aspect='auto', cmap='viridis')
        ax.set_title(f'Statistical Matrix for {sub}_{output_name_event}')
        ax.set_xlabel('Samples (204.8 Hz after decimation, 0 is -1 sec)')
        ax.set_ylabel('Channels')
        # plt.show()

        # Save the figure
        plot_save_path = os.path.join(save_dir, f'{output_name_event}_stats.png')
        fig.savefig(plot_save_path, dpi=300)

        channels = good.ch_names

        #save channels with their indices 
        save_channels_to_file(channels, sub, task, save_dir)

        # save significant channels to a json
        save_sig_chans(f'{output_name_event}', mat, channels, sub, save_dir)

        # sig_chans_filename = os.path.join(save_dir, f'sig_chans_{sub}_{output_name}.json')
        # sig_chans = load_sig_chans(sig_chans_filename)

        # Assuming `mat` is your array and `save_dir` is the directory where you want to save it
        mat_save_path = os.path.join(save_dir, f'{output_name_event}_mat.npy')

        # Save the mat array
        np.save(mat_save_path, mat)

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

plot the time perm cluster results outside of the function cuz it seems broken right now 6/6

In [37]:
LAB_root = None

task='GlobalLocal'
# output_name_event = "Stimulus_1sec_preStimulusBase_decFactor_10"
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107']
if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)

for sub in subjects:
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    # Load the matrix
    mat_save_path = os.path.join(save_dir, f'{output_name_event}_mat.npy')
    mat = np.load(mat_save_path)
    # Plot the matrix as one figure
    fig, ax = plt.subplots()
    cax = ax.imshow(mat, aspect='auto', cmap='viridis')
    ax.set_title(f'Statistical Matrix for {sub}_{output_name_event}')
    ax.set_xlabel('Samples (204.8 Hz after decimation, 0 is -1 sec)')
    ax.set_ylabel('Channels')
    # fig.colorbar(cax)
    # plt.show()

    # Save the figure
    plot_save_path = os.path.join(save_dir, f'{output_name_event}_stats.png')
    fig.savefig(plot_save_path, dpi=300)

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected


...

Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (170) does not match the number of channels in the raw data file (169). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0107\ieeg\sub-D0107_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (170) does not match the number of channels in the raw data file (169). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Not fully anonymizing info - keeping his_id, sex, and hand info


5/2 do this for each subject, for stimulus and response

In [156]:
times = (-1,1.5)
within_base_times = (-1,0)
base_times_length = 0.5
pad_length = 0.5

# subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107']
# subjects = ['D0100', 'D0102', 'D0103', 'D0107']
subjects = ['D0057']
for sub in subjects:
    plot_HG_and_stats(sub=sub, task='GlobalLocal', times=times,
                      within_base_times=within_base_times, base_times_length = 0.5, pad_length = 0.5, dec_factor=8)

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


good channels before dropping bads: 178
filt channels before dropping bads: 178
outlier round 1 channels: ['RAMT8']
outlier round 2 channels: ['RAMT8', 'RPI16']
outlier round 2 channels: ['RAMT8', 'RPI16', 'LAMT12']
Bad channels in 'good': ['RAMT8', 'RPI16', 'LAMT12']
Bad channels in 'good' after dropping once: []
good channels after dropping bads: 175
filt channels after dropping bads: 175
Reading 0 ... 3433906  =      0.000 ...  1676.712 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['1', '2', 'Response/c25.0/n25.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount225.0/BlockTrialCount1.0/ReactionTime2149.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime883.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/

C:\Users\jz421\AppData\Local\Temp\ipykernel_3924\359444317.py:58: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for trial in trials.get_data():
100%|██████████| 449/449 [02:02<00:00,  3.67it/s]


IndexError: boolean index did not match indexed array along dimension 2; dimension is 3072 but corresponding boolean dimension is 4097

time shuffled baseline vs non-shuffled plotting 6/4

In [142]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming HG_base_reshaped and test_sig are 3D numpy arrays with shape (trials, channels, time)

# Average across trials (axis=0) and channels (axis=1)
randoffset_base = mne.read_epochs(r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_preStimulusBase_decFactor_8_HG_base-epo.fif')
# old_base = mne.read_epochs(r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_1sec_preStimulusBase_decFactor_10_HG_base-epo.fif')
randoffset_base_data = randoffset_base.get_data()
# old_base_data = old_base.get_data()

shuffled_avg = np.nanmean(randoffset_base_data, axis=(0, 1))
# unshuffled_avg = np.nanmean(old_base_data, axis=(0, 1))

# Create a time axis based on the shape of the time dimension
time_points = np.arange(shuffled_avg.shape[0])

# Plot the averaged data
plt.figure(figsize=(10, 5))
plt.plot(time_points, shuffled_avg, label='Averaged rand offset baseline')
# plt.plot(time_points, unshuffled_avg, label='Averaged unshuffled baseline')
plt.xlabel('Samples (256 Hz after decimation)')
plt.ylabel('Amplitude')
plt.title('Averaged rand offset and unshuffled baseline Across Trials and Channels')
plt.legend()
plt.show()


Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_preStimulusBase_decFactor_8_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    -500.00 ms
        0 CTF compensation matrices available


ValueError: Incorrect number of samples (10113600 instead of 10136175)

turn this into a loop over all three time windows and all 12 subjects

In [ ]:
from PIL import Image, ImageChops

def trim_whitespace(image):
    """
    Trims the whitespace from an image.
    """
    bg = Image.new(image.mode, image.size, image.getpixel((0, 0)))
    diff = ImageChops.difference(image, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return image.crop(bbox)
    return image  # If no change

def plot_channels_on_grid_windows(evoke_data, std_err_data, channels_subset, time_windows, sig_chans, sample_rate, plot_x_dim=6, plot_y_dim=6):
    """
    Plots evoked EEG/MEG data for a subset of channels on a grid, overlaying significance markers for specified time windows.

    Parameters:
    - evoke_data: mne.Evoked object
        The evoked data to be plotted. This object contains the averaged EEG/MEG data over epochs.
    - std_err_data: 
        The standard error of the evoked data to be plotted
    - channels_subset: list of str
        A list of channel names to be plotted. Each channel name must correspond to a channel in `evoke_data`.
    - time_windows: dict
        A dictionary where keys are strings representing the names of the time windows of interest, and values are tuples
        indicating the start and end indices (in samples) of these windows.
    - sig_chans: dict
        A dictionary where keys are the names of the time windows (matching those in `time_windows`) and values are lists
        of channel names (str) that are significant within those windows.
    - sample_rate: float
        The sampling rate of the data, in Hz. Used to convert sample indices in `time_windows` to time in seconds.
    - plot_x_dim: int, optional (default=6)
        The number of columns in the grid layout for plotting the channels.
    - plot_y_dim: int, optional (default=6)
        The number of rows in the grid layout for plotting the channels.

    Returns:
    - fig: matplotlib.figure.Figure object
        The figure object containing the grid of plots. Each plot shows the evoked data for a channel, with significance
        markers overlaid for the specified time windows.
    """
    fig, axes = plt.subplots(plot_y_dim, plot_x_dim, figsize=(20, 12))  # Adjusted to match your desired layout
    fig.suptitle("Channels with Significance Overlay for Different Time Windows")
    axes_flat = axes.flatten()

    # Define colors for each time window
    colors = ['red', 'green', 'blue']
    window_names = list(time_windows.keys())

    for channel, ax in zip(channels_subset, axes_flat):
        stderr = stderr_data.data[channel_to_index[channel], :]
        # Plot the channel data with times in seconds
        ax.plot(evoke_data.times, evoke_data.data[channel_to_index[channel], :])
         # Add the standard error shading
        ax.fill_between(evoke_data.times, evoke_data.data[channel_to_index[channel], :] - stderr, evoke_data.data[channel_to_index[channel], :] + stderr, alpha=0.2)

        max_y_value = np.max(evoke_data.data[channel_to_index[channel], :])  # Find max y-value for significance lines
        # Assuming the epochs start 1 second before the stimulus/event
        epoch_start_time = -1  # Start time of epochs in seconds

        for window_index, window_name in enumerate(window_names):
            if channel in sig_chans[window_name]:
                start_idx, end_idx = time_windows[window_name]
                # Convert sample indices to times in seconds
                start_time = (start_idx / sample_rate) + epoch_start_time
                end_time = (end_idx / sample_rate) + epoch_start_time
                # Determine y-position for the significance line, adjusting to avoid overlap
                y_position = max_y_value - (window_index * 0.02 * max_y_value)  # Adjust overlap offset here

                # Cycle through colors for each time window
                color = colors[window_index % len(colors)]
                ax.hlines(y=y_position, xmin=start_time, xmax=end_time, color=color, linewidth=2, label=f"{window_name}: {color}")

        ax.set_title(channel)

    # Create a legend for the first subplot (if desired) to explain the colors
    if len(axes_flat) > 0 and len(window_names) > 0:
        handles, labels = axes_flat[0].get_legend_handles_labels()
        fig.legend(handles, labels, loc='upper right', title="Time Windows & Colors")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make space for the legend
    return fig

sig_chans = {}

for sub in subjects:
    task = 'GlobalLocal'
    LAB_root = None
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    sample_rate = filt.info['sfreq'] # get sampling rate, should be 2048 Hz
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    time_windows = {
        "Stimulus_fixationCrossBase_0.2sec_window_0to0.5": (sample_rate,1.5*sample_rate), #actually grab from 1 to 1.5 because the epochs start at -1 second before stim onset
        "Stimulus_fixationCrossBase_0.2sec_window_0.5to1": (1.5*sample_rate,2*sample_rate),
        "Stimulus_fixationCrossBase_0.2sec_window_0to1": (sample_rate,2*sample_rate)
    }

    for window in time_windows:
        output_name = window

        # Define file paths
        HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
        HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
        HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

        # Load the epochs and evoked objects
        HG_ev1 = mne.read_epochs(HG_ev1_file)
        HG_base = mne.read_epochs(HG_base_file)
        HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
        HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
        HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))
        HG_ev1_evoke_stderr = HG_ev1.standard_error()
        HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

        channels = [] # load in all channels
        channel_to_index = {}
        channel_file = os.path.join(save_dir, f'channels_{sub}_GlobalLocal.txt') 
        with open(channel_file, 'r') as f:
            for line in f:
                index, channel = line.strip().split(': ')
                channels.append(channel)
                channel_to_index[channel] = int(index)

        sig_chans_filename = os.path.join(save_dir, f'sig_chans_{sub}_{output_name}.json') # load in sig channels
        sig_chans[window] = load_sig_chans(sig_chans_filename)

    # now plot 6x6 grid of 36 channels on one plot, for the z-scored signal
    plot_x_dim = 6
    plot_y_dim = 6
    channels_per_fig = plot_x_dim * plot_y_dim

    # Iterate over all channels in chunks and plot them with z-scored signal
    for i in range(0, len(channels), channels_per_fig):
        channels_subset = channels[i:i+channels_per_fig]
        fig = plot_channels_on_grid_windows(HG_ev1_evoke_rescaled, HG_ev1_evoke_rescaled_stderr, channels_subset, time_windows, sig_chans, sample_rate, plot_x_dim, plot_y_dim)
        combined_plot_path_rescaled = os.path.join(save_dir, f'{sub}_zscore_{output_name}_combinedChannelTracesAndWindowsSignificance_Page_{i//channels_per_fig + 1}.png')
        fig.savefig(combined_plot_path_rescaled)
        plt.close(fig)

    for i in range(0, len(channels), channels_per_fig):
        channels_subset = channels[i:i+channels_per_fig]
        fig = plot_channels_on_grid_windows(HG_ev1_evoke, HG_ev1_evoke_stderr, channels_subset, time_windows, sig_chans, sample_rate, plot_x_dim, plot_y_dim)
        combined_plot_path_rescaled = os.path.join(save_dir, f'{sub}_raw_{output_name}_combinedChannelTracesAndWindowsSignificance_Page_{i//channels_per_fig + 1}.png')
        fig.savefig(combined_plot_path_rescaled)
        plt.close(fig)

this below code is for when using the time perm cluster stats to determine significance timepoint by timepoint

In [ ]:
# UNTESTED 2/29, NEED TO RERUN STATS TO SAVE MAT FIRST.
def plot_channels_on_grid_time_perm_cluster(evoke_data, std_err_data, channels_subset, mat, sample_rate=2048, plot_x_dim=6, plot_y_dim=6):
    """
    Plots evoked EEG/MEG data for a subset of channels on a grid, overlaying significance markers for specified time windows.

    Parameters:
    - evoke_data: mne.Evoked object
        The evoked data to be plotted. This object contains the averaged EEG/MEG data over epochs.
    - std_err_data: 
        The standard error of the evoked data to be plotted
    - channels_subset: list of str
        A list of channel names to be plotted. Each channel name must correspond to a channel in `evoke_data`.
    - mat: numpy.array
        A binary matrix (same shape as evoke_data) indicating significant data points (1 for significant, 0 for non-significant).
    - sample_rate: float
    - sample_rate: float
        The sampling rate of the data, in Hz. Used to convert sample indices in `time_windows` to time in seconds.
    - plot_x_dim: int, optional (default=6)
        The number of columns in the grid layout for plotting the channels.
    - plot_y_dim: int, optional (default=6)
        The number of rows in the grid layout for plotting the channels.

    Returns:
    - fig: matplotlib.figure.Figure object
        The figure object containing the grid of plots. Each plot shows the evoked data for a channel, with significance
        markers overlaid for the specified time windows.
    """
    fig, axes = plt.subplots(plot_x_dim, plot_y_dim, figsize=(20, 12))
    fig.suptitle("Channels with Significance Overlay")
    axes_flat = axes.flatten()

    for channel, ax in zip(channels_subset, axes_flat):
        stderr = std_err_data.data[channel_to_index[channel], :]
        time_in_seconds = np.arange(0, len(mat[channel_to_index[channel]])) / sample_rate  # Should be 2048 Hz sample rate
        sig_data_in_seconds = np.array(mat[channel_to_index[channel]])
        ax.plot(evoke_data.times, evoke_data.data[channel_to_index[channel], :])
         # Add the standard error shading
        ax.fill_between(evoke_data.times, evoke_data.data[channel_to_index[channel], :] - stderr, evoke_data.data[channel_to_index[channel], :] + stderr, alpha=0.2)

        # Find the maximum y-value for the current channel
        max_y_value = np.max(evoke_data.data[channel_to_index[channel], :])

        # Overlay significance as a horizontal line at the max y-value
        significant_points = np.where(sig_data_in_seconds == 1)[0]
        for point in significant_points:
            ax.hlines(y=max_y_value, xmin=time_in_seconds[point]-1, xmax=time_in_seconds[point] + 0.005 - 1, color='red', linewidth=1) # subtract 1 cuz the sig time is from 0 to 2.5, while the high gamma time is from -1 to 1.5

        ax.set_title(channel)

    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    return fig

plot_x_dim = 6
plot_y_dim = 6
channels_per_fig = plot_x_dim * plot_y_dim

sig_chans = {}

for sub in subjects:
    task = 'GlobalLocal'
    LAB_root = None
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    sample_rate = filt.info['sfreq'] # get sampling rate, should be 2048 Hz
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    output_name = 'Stimulus_fixationCrossBase_1sec_mirror_0to1test'

    # Define file paths
    HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
    HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
    HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

    # Load the epochs and evoked objects
    HG_ev1 = mne.read_epochs(HG_ev1_file)
    HG_base = mne.read_epochs(HG_base_file)
    HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_stderr = HG_ev1.standard_error()
    HG_ev1_evoke_rescaled_stderr = HG_ev1_rescaled.standard_error()

    mat_save_path = os.path.join(save_dir, f'{output_name}_mat.npy')
    mat = np.load(mat_save_path)

    channels = [] # load in all channels
    channel_to_index = {}
    channel_file = os.path.join(save_dir, f'channels_{sub}_GlobalLocal.txt') 
    with open(channel_file, 'r') as f:
        for line in f:
            index, channel = line.strip().split(': ')
            channels.append(channel)
            channel_to_index[channel] = int(index)
    
    # Iterate over all channels in chunks of channels_per_fig (plot_x_dim * plot_y_dim) and plot them
    for i in range(0, len(channels), channels_per_fig):
        channels_subset = channels[i:i+channels_per_fig]
        fig = plot_channels_on_grid_time_perm_cluster(HG_ev1_evoke_rescaled, HG_ev1_evoke_rescaled_stderr, channels_subset, mat, plot_x_dim, plot_y_dim, sample_rate=sample_rate)
        combined_plot_path = os.path.join(save_dir, f'{sub}_zscore_{output_name}_combinedChannelTracesAndTimePermClusterSignificance_Page_{i//channels_per_fig + 1}.png')
        fig.savefig(combined_plot_path)
        plt.close(fig)

        # Iterate over all channels in chunks of channels_per_fig (plot_x_dim * plot_y_dim) and plot them
    for i in range(0, len(channels), channels_per_fig):
        channels_subset = channels[i:i+channels_per_fig]
        fig = plot_channels_on_grid_time_perm_cluster(HG_ev1_evoke, HG_ev1_evoke_stderr, channels_subset, mat, plot_x_dim, plot_y_dim, sample_rate=sample_rate)
        combined_plot_path = os.path.join(save_dir, f'{sub}_raw_{output_name}_combinedChannelTracesAndTimePermClusterSignificance_Page_{i//channels_per_fig + 1}.png')
        fig.savefig(combined_plot_path)
        plt.close(fig)

### z-scored signal

In [ ]:
# for raw traces, just plot HG_ev1_evoke instead of HG_ev1_evoke_rescaled. And for the standard error, use the HG_ev1_evoke_stderr instead of HG_ev1_evoke_rescaled_stderr

### raw traces

In [ ]:
# Assuming all imports and previous definitions are in place

def plot_channels_on_grid_time_perm_cluster_raw(channels_subset):
    fig, axes = plt.subplots(6, 10, figsize=(20, 33))
    fig.suptitle("Channels with Significance Overlay")
    axes_flat = axes.flatten()

    for channel, ax in zip(channels_subset, axes_flat):
        stderr = HG_ev1_evoke_stderr.data[channel_to_index[channel], :]
        time_in_seconds = np.arange(0, len(mat[channel_to_index[channel]])) / sample_rate  # should be 2048 Hz sample rate. Need mat though..should i save this somehow?
        sig_data_in_seconds = np.array(mat[channel_to_index[channel]])
        ax.plot(HG_ev1_evoke.times, HG_ev1_evoke.data[channel_to_index[channel], :])
         # Add the standard error shading
        ax.fill_between(HG_ev1_evoke.times, HG_ev1_evoke.data[channel_to_index[channel], :] - stderr, HG_ev1_evoke.data[channel_to_index[channel], :] + stderr, alpha=0.2)

        # Find the maximum y-value for the current channel
        max_y_value = np.max(HG_ev1_evoke.data[channel_to_index[channel], :])

        # Overlay significance as a horizontal line at the max y-value
        significant_points = np.where(sig_data_in_seconds == 1)[0]
        for point in significant_points:
            ax.hlines(y=max_y_value, xmin=time_in_seconds[point]-1, xmax=time_in_seconds[point] + 0.005 - 1, color='red', linewidth=1) # subtract 1 cuz the sig time is from 0 to 2.5, while the high gamma time is from -1 to 1.5

        ax.set_title(channel)

    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    return fig

# Iterate over all channels in chunks of 60 and plot them
for i in range(0, len(sig_chans), 60):
    channels_subset = channels[i:i+60]
    fig = plot_channels_on_grid_time_perm_cluster_raw(channels_subset)
    combined_plot_path = os.path.join(save_dir, f'{sub}_raw_{output_name}_combinedChannelTracesAndTimePermClusterSignificance_Page_{i//36 + 1}.png')
    fig.savefig(combined_plot_path)
    plt.close(fig)
